In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
import json
from tqdm.notebook import tqdm

In [ ]:
class RD:
    def __init__(self, api_token: str):
        self.headers = {"Authorization": f'Bearer {api_token}'}
        self.tup_data = list()
        self.savepath = ''
        self.success = list()
    
    def tup(self, fpath):
        r = requests.put('https://api.real-debrid.com/rest/1.0/torrents/addTorrent', data=open(fpath, 'rb'), headers=self.headers)
        if r.ok:
            return r.json()
    
    def ttup(self, fpaths):
        with ThreadPoolExecutor(max_workers=20) as executor:
            futures = executor.map(self.tup, fpaths)
            for future in futures:
                self.tup_data.append(future)

    def get_tids(self):
        response = requests.get(f'https://api.real-debrid.com/rest/1.0/torrents', headers=self.headers)
        data = response.json()
        return data

    def get_tinfo(self, tid):
        response = requests.get(f'https://api.real-debrid.com/rest/1.0/torrents/info/{tid}?', headers=self.headers)
        data = response.json()
        return data.json()

    def dltor(self, data):
        if data.get('status') == 'downloaded':
                link_url = data.get('links')
                params = {'link': link_url }
                response = requests.post('https://api.real-debrid.com/rest/1.0/unrestrict/link', headers=self.headers, data=params)
                d = response.json()

                fname = data.get('filename')
                totalfilename = os.path.join(self.savepath, fname)
                total_length = data.get('bytes')
                chunkSize = 16 * 1024
                dlink = d.get('download')
                r = requests.get(dlink, stream=True)
                with open(totalfilename, 'wb+') as f:
                    for i in tqdm(iterable = r.iter_content(chunk_size= chunkSize), total=total_length/chunkSize, unit = 'KB', desc=fname):
                        f.write(i)
                print(r)
                return r
                
        return data.get('filename')
    
    def t_dltor(self, datas):
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = executor.map(self.dltor, datas)
            for future in futures:
                self.success.append(future)

    def r_tor(self, tid):
        response = requests.delete(f'https://api.real-debrid.com/rest/1.0/torrents/delete/{tid}', headers=self.headers)
        return response


In [ ]:
api_token = ''
rd = RD(api_token)
rd.savepath = r'C:\\mydocs'
tids = rd.get_tids()
for i in tids:
    print(i.get('filename'))